In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

from nn.neural_network import NeuralNetwork
from nn.layers import Input, Convolutional, Reshape, Dense

In [ ]:
def preprocess_data(x, y, limit):
    indexes = []
    n = 10
    for i in range(n):
        indexes.append(np.where(y == i)[0][:limit])

    all_indices = np.hstack(indexes)
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), n, 1)
    return x, y

In [ ]:
# get dataset and split it into two datasets (training, testing)
np.random.seed(15)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# preprocessing the data
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 10)

# set of images
images = x_test.copy()

In [ ]:
# build the convolutional neural network
model = NeuralNetwork(loss_f='MSE')

# adding layers
model.add_layer(Input(input_shape=x_train.shape[1:]))
model.add_layer(Convolutional(kernel_size=3, depth=3, act_f='sigmoid'))
model.add_layer(Convolutional(kernel_size=4, depth=1, act_f='sigmoid'))
model.add_layer(Reshape())
model.add_layer(Dense(n_neur=30, act_f='sigmoid'))
model.add_layer(Dense(n_neur=20, act_f='sigmoid'))
model.add_layer(Dense(n_neur=10, act_f='sigmoid'))

# get the summary of the model
model.summary()

In [ ]:
# training
loss = model.fit(x_train, y_train, epochs=600, lr=0.01)

# plot the loss curve
plt.figure(figsize=(10, 6))
plt.plot(list(range(len(loss))), loss)
plt.title('Loss curve')
plt.show()

In [ ]:
# evaluation
yp = model.predict(x_test)
count = 0
for i in range(len(yp)):
    pred = np.argmax(yp[i])
    true = np.argmax(y_test[i])
    if pred == true: count = count + 1

# print the accuracy of the model
print(f'Accuracy: {count / len(yp) * 100}%')

In [ ]:
# show images and their predicted class
fig, axes = plt.subplots(10,10, figsize=(20,20))
fig.tight_layout()
for i,ax in enumerate(axes.flat):
    ax.imshow(images[i][0])
    ax.title.set_text(f'Prediction: {np.argmax(yp[i])}')